# Introduction

This notebook presents **LSTM** network with character-wise input trained on Shakespeare plays.

Dataset file is included in this repo and consists of all works of Shakespeare concatenated together (4.6MB).

# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import collections

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Read Data

Dataset location

In [4]:
dataset_location = '../Datasets/shakespeare/shakespeare_input.txt'

Open text file

In [5]:
with open(dataset_location, 'r') as f:
    text = f.read()
print(text[:173])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.


Discard bit at the end such that text is divisible by 1024. This allow for batch sizes [1, 2, 4, 8, 16, 32, ..., 1024]

In [6]:
mod1024 = len(text) % 1024
text = text[:-mod1024]

Tokenize

In [7]:
tokens = collections.Counter(text).most_common()
tokens[0:5]

[(' ', 696165), ('e', 386342), ('t', 272672), ('o', 268956), ('a', 230593)]

In [8]:
i2c = {i : c for i, (c, n) in enumerate(tokens)}
c2i = {c : i for i, c in i2c.items()}
print('i2c:', i2c)
print('c2i:', c2i)

i2c: {0: ' ', 1: 'e', 2: 't', 3: 'o', 4: 'a', 5: 'h', 6: 's', 7: 'n', 8: 'r', 9: 'i', 10: '\n', 11: 'l', 12: 'd', 13: 'u', 14: 'm', 15: 'y', 16: ',', 17: 'w', 18: 'f', 19: 'c', 20: 'g', 21: 'I', 22: ':', 23: 'b', 24: 'p', 25: 'A', 26: '.', 27: 'v', 28: 'T', 29: 'k', 30: "'", 31: 'S', 32: 'E', 33: 'O', 34: 'N', 35: 'R', 36: 'L', 37: ';', 38: 'C', 39: 'H', 40: 'W', 41: 'M', 42: 'U', 43: 'B', 44: 'D', 45: '?', 46: 'F', 47: '!', 48: '-', 49: 'G', 50: 'P', 51: 'Y', 52: 'K', 53: 'V', 54: 'j', 55: 'q', 56: 'x', 57: 'J', 58: 'z', 59: 'Q', 60: 'Z', 61: 'X', 62: '3', 63: '&', 64: '[', 65: ']', 66: '$'}
c2i: {' ': 0, 'e': 1, 't': 2, 'o': 3, 'a': 4, 'h': 5, 's': 6, 'n': 7, 'r': 8, 'i': 9, '\n': 10, 'l': 11, 'd': 12, 'u': 13, 'm': 14, 'y': 15, ',': 16, 'w': 17, 'f': 18, 'c': 19, 'g': 20, 'I': 21, ':': 22, 'b': 23, 'p': 24, 'A': 25, '.': 26, 'v': 27, 'T': 28, 'k': 29, "'": 30, 'S': 31, 'E': 32, 'O': 33, 'N': 34, 'R': 35, 'L': 36, ';': 37, 'C': 38, 'H': 39, 'W': 40, 'M': 41, 'U': 42, 'B': 43, 'D': 44

Encode text as tokens, reshape to batches, convert to tensor

In [9]:
batch_size = 128

data = np.array([c2i[c] for c in text])
data = data.reshape((batch_size, -1))
print('data:')
print(data)
print('shape:', data.shape)

data:
[[46  9  8 ...  3  2  0]
 [ 9  7  0 ... 30  6 16]
 [ 0 17  4 ...  1 19  3]
 ...
 [43  8  3 ...  0  9  2]
 [ 0  9  7 ...  5  1  8]
 [ 0  2  5 ...  0  4  8]]
shape: (128, 35728)


In [10]:
split_index = int(data.shape[1]*.9)  # 90% train, 10% valid
train_data, valid_data = np.split(data, [split_index], axis=1)
print('train_data:', train_data.shape)
print('valid_data:', valid_data.shape)

train_data: (128, 32155)
valid_data: (128, 3573)


Move to GPU if possible

In [11]:
train_x = torch.tensor(train_data).to(device)
valid_x = torch.tensor(valid_data).to(device)
print('train_x:', train_x.shape)
print('valid_x:', valid_x.shape)

train_x: torch.Size([128, 32155])
valid_x: torch.Size([128, 3573])


Model

In [12]:
class CharRNN(nn.Module):
    
    def __init__(self, nb_layers, n_in, n_embed, n_hid, n_out, dropout):
        super(CharRNN, self).__init__()
        self.embed = nn.Embedding(num_embeddings=n_in, embedding_dim=n_embed)
        self.lstm = nn.LSTM(input_size=n_embed, hidden_size=n_hid, num_layers=nb_layers,
                           batch_first=True, dropout=dropout)
        self.drop = nn.Dropout(p=dropout)
        self.fc = nn.Linear(in_features=n_hid, out_features=n_out)
    
    def forward(self, x, hidden):
        x = self.embed(x)                   # shape [n_batch, n_seq, n_embed]
        x, hidden = self.lstm(x, hidden)    # shape [n_batch, n_seq, n_hid]
        x = self.drop(x)
        x = self.fc(x)                      # shape [n_batch, n_seq, n_out]
        return x, hidden

In [13]:
nb_layers = 2
n_in = len(i2c)
n_seq = 256
n_embed = 50
n_hid = 64
n_out = len(i2c)
dropout = .5

model = CharRNN(nb_layers, n_in, n_embed, n_hid, n_out, dropout)
model.to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [18]:
import time

In [ ]:
trace = {'tloss': []}

In [26]:


ts = time.time()
for e in range(18):
    
    ### Train ###
    model.train()
    hidden = None  # reset LSTM hidden state
    for i in range(0, train_x.shape[1]-1, n_seq):
        
        # Pick mini-batch (over seqence dimension)
        inputs = train_x[:,i:i+n_seq]        # shape [n_batch, n_seq], less for last batch
        targets = train_x[:,i+1:i+1+n_seq]   # shape [n_batch, n_seq], less for last batch
        if inputs.shape[1] != targets.shape[1]:
            inputs = inputs[:,:-1]           # fix shape disparity for last batch in epoch
        
        assert inputs.shape == targets.shape
        
        # Optimize
        optimizer.zero_grad()
        outputs, hidden = model(inputs, hidden)
        hidden = tuple(h.detach() for h in hidden)
        loss = criterion(outputs.view(-1, n_out), targets.flatten())
        loss.backward()
        optimizer.step()

        # Record per-iteration loss
        trace['tloss'].append( loss.item() )
        
        print(f'Epoch: {e:3} {i*100/train_x.shape[1]:4.1f}%     loss: {loss.item():.4f}')
        
        

print(time.time() - ts)

Epoch:   0  0.0%     loss: 2.6072
Epoch:   0  0.8%     loss: 2.5926
Epoch:   0  1.6%     loss: 2.5955
Epoch:   0  2.4%     loss: 2.5999
Epoch:   0  3.2%     loss: 2.5891
Epoch:   0  4.0%     loss: 2.5904
Epoch:   0  4.8%     loss: 2.5862
Epoch:   0  5.6%     loss: 2.5794
Epoch:   0  6.4%     loss: 2.5813
Epoch:   0  7.2%     loss: 2.5836
Epoch:   0  8.0%     loss: 2.5808
Epoch:   0  8.8%     loss: 2.5847
Epoch:   0  9.6%     loss: 2.5761
Epoch:   0 10.3%     loss: 2.5840
Epoch:   0 11.1%     loss: 2.5723
Epoch:   0 11.9%     loss: 2.5817
Epoch:   0 12.7%     loss: 2.5893
Epoch:   0 13.5%     loss: 2.5722
Epoch:   0 14.3%     loss: 2.5752
Epoch:   0 15.1%     loss: 2.5811
Epoch:   0 15.9%     loss: 2.5741
Epoch:   0 16.7%     loss: 2.5726
Epoch:   0 17.5%     loss: 2.5694
Epoch:   0 18.3%     loss: 2.5590
Epoch:   0 19.1%     loss: 2.5773
Epoch:   0 19.9%     loss: 2.5552
Epoch:   0 20.7%     loss: 2.5600
Epoch:   0 21.5%     loss: 2.5393
Epoch:   0 22.3%     loss: 2.5505
Epoch:   0 23.

Epoch:   1 93.9%     loss: 2.4005
Epoch:   1 94.7%     loss: 2.4004
Epoch:   1 95.5%     loss: 2.3736
Epoch:   1 96.3%     loss: 2.3770
Epoch:   1 97.1%     loss: 2.3701
Epoch:   1 97.9%     loss: 2.3904
Epoch:   1 98.7%     loss: 2.3903
Epoch:   1 99.5%     loss: 2.3952
Epoch:   2  0.0%     loss: 2.3905
Epoch:   2  0.8%     loss: 2.3675
Epoch:   2  1.6%     loss: 2.3922
Epoch:   2  2.4%     loss: 2.3833
Epoch:   2  3.2%     loss: 2.3767
Epoch:   2  4.0%     loss: 2.3791
Epoch:   2  4.8%     loss: 2.3837
Epoch:   2  5.6%     loss: 2.3672
Epoch:   2  6.4%     loss: 2.3664
Epoch:   2  7.2%     loss: 2.3789
Epoch:   2  8.0%     loss: 2.3744
Epoch:   2  8.8%     loss: 2.3750
Epoch:   2  9.6%     loss: 2.3713
Epoch:   2 10.3%     loss: 2.3732
Epoch:   2 11.1%     loss: 2.3661
Epoch:   2 11.9%     loss: 2.3753
Epoch:   2 12.7%     loss: 2.3821
Epoch:   2 13.5%     loss: 2.3757
Epoch:   2 14.3%     loss: 2.3775
Epoch:   2 15.1%     loss: 2.3797
Epoch:   2 15.9%     loss: 2.3686
Epoch:   2 16.

Epoch:   3 85.2%     loss: 2.2710
Epoch:   3 86.0%     loss: 2.2688
Epoch:   3 86.8%     loss: 2.2668
Epoch:   3 87.6%     loss: 2.2778
Epoch:   3 88.4%     loss: 2.2780
Epoch:   3 89.2%     loss: 2.2841
Epoch:   3 90.0%     loss: 2.2777
Epoch:   3 90.8%     loss: 2.2659
Epoch:   3 91.6%     loss: 2.2793
Epoch:   3 92.4%     loss: 2.2897
Epoch:   3 93.1%     loss: 2.2928
Epoch:   3 93.9%     loss: 2.2959
Epoch:   3 94.7%     loss: 2.2895
Epoch:   3 95.5%     loss: 2.2733
Epoch:   3 96.3%     loss: 2.2849
Epoch:   3 97.1%     loss: 2.2620
Epoch:   3 97.9%     loss: 2.2886
Epoch:   3 98.7%     loss: 2.2929
Epoch:   3 99.5%     loss: 2.2965
Epoch:   4  0.0%     loss: 2.2923
Epoch:   4  0.8%     loss: 2.2685
Epoch:   4  1.6%     loss: 2.2883
Epoch:   4  2.4%     loss: 2.2831
Epoch:   4  3.2%     loss: 2.2712
Epoch:   4  4.0%     loss: 2.2773
Epoch:   4  4.8%     loss: 2.2825
Epoch:   4  5.6%     loss: 2.2633
Epoch:   4  6.4%     loss: 2.2654
Epoch:   4  7.2%     loss: 2.2759
Epoch:   4  8.

Epoch:   5 78.8%     loss: 2.1988
Epoch:   5 79.6%     loss: 2.2249
Epoch:   5 80.4%     loss: 2.2328
Epoch:   5 81.2%     loss: 2.2248
Epoch:   5 82.0%     loss: 2.2289
Epoch:   5 82.8%     loss: 2.2127
Epoch:   5 83.6%     loss: 2.2079
Epoch:   5 84.4%     loss: 2.2103
Epoch:   5 85.2%     loss: 2.2000
Epoch:   5 86.0%     loss: 2.1900
Epoch:   5 86.8%     loss: 2.1996
Epoch:   5 87.6%     loss: 2.2123
Epoch:   5 88.4%     loss: 2.2084
Epoch:   5 89.2%     loss: 2.2126
Epoch:   5 90.0%     loss: 2.2063
Epoch:   5 90.8%     loss: 2.1975
Epoch:   5 91.6%     loss: 2.2108
Epoch:   5 92.4%     loss: 2.2267
Epoch:   5 93.1%     loss: 2.2260
Epoch:   5 93.9%     loss: 2.2242
Epoch:   5 94.7%     loss: 2.2255
Epoch:   5 95.5%     loss: 2.2042
Epoch:   5 96.3%     loss: 2.2186
Epoch:   5 97.1%     loss: 2.1985
Epoch:   5 97.9%     loss: 2.2222
Epoch:   5 98.7%     loss: 2.2202
Epoch:   5 99.5%     loss: 2.2287
Epoch:   6  0.0%     loss: 2.2309
Epoch:   6  0.8%     loss: 2.2022
Epoch:   6  1.

Epoch:   7 70.9%     loss: 2.1713
Epoch:   7 71.7%     loss: 2.1693
Epoch:   7 72.4%     loss: 2.1630
Epoch:   7 73.2%     loss: 2.1593
Epoch:   7 74.0%     loss: 2.1557
Epoch:   7 74.8%     loss: 2.1652
Epoch:   7 75.6%     loss: 2.1584
Epoch:   7 76.4%     loss: 2.1743
Epoch:   7 77.2%     loss: 2.1792
Epoch:   7 78.0%     loss: 2.1615
Epoch:   7 78.8%     loss: 2.1524
Epoch:   7 79.6%     loss: 2.1786
Epoch:   7 80.4%     loss: 2.1767
Epoch:   7 81.2%     loss: 2.1820
Epoch:   7 82.0%     loss: 2.1725
Epoch:   7 82.8%     loss: 2.1642
Epoch:   7 83.6%     loss: 2.1633
Epoch:   7 84.4%     loss: 2.1540
Epoch:   7 85.2%     loss: 2.1480
Epoch:   7 86.0%     loss: 2.1432
Epoch:   7 86.8%     loss: 2.1575
Epoch:   7 87.6%     loss: 2.1607
Epoch:   7 88.4%     loss: 2.1606
Epoch:   7 89.2%     loss: 2.1660
Epoch:   7 90.0%     loss: 2.1544
Epoch:   7 90.8%     loss: 2.1518
Epoch:   7 91.6%     loss: 2.1547
Epoch:   7 92.4%     loss: 2.1694
Epoch:   7 93.1%     loss: 2.1832
Epoch:   7 93.

Epoch:   9 62.1%     loss: 2.1293
Epoch:   9 62.9%     loss: 2.1401
Epoch:   9 63.7%     loss: 2.1299
Epoch:   9 64.5%     loss: 2.1236
Epoch:   9 65.3%     loss: 2.1311
Epoch:   9 66.1%     loss: 2.1264
Epoch:   9 66.9%     loss: 2.1374
Epoch:   9 67.7%     loss: 2.1375
Epoch:   9 68.5%     loss: 2.1258
Epoch:   9 69.3%     loss: 2.1536
Epoch:   9 70.1%     loss: 2.1456
Epoch:   9 70.9%     loss: 2.1241
Epoch:   9 71.7%     loss: 2.1320
Epoch:   9 72.4%     loss: 2.1260
Epoch:   9 73.2%     loss: 2.1157
Epoch:   9 74.0%     loss: 2.1241
Epoch:   9 74.8%     loss: 2.1222
Epoch:   9 75.6%     loss: 2.1258
Epoch:   9 76.4%     loss: 2.1356
Epoch:   9 77.2%     loss: 2.1384
Epoch:   9 78.0%     loss: 2.1217
Epoch:   9 78.8%     loss: 2.1145
Epoch:   9 79.6%     loss: 2.1392
Epoch:   9 80.4%     loss: 2.1463
Epoch:   9 81.2%     loss: 2.1365
Epoch:   9 82.0%     loss: 2.1364
Epoch:   9 82.8%     loss: 2.1246
Epoch:   9 83.6%     loss: 2.1229
Epoch:   9 84.4%     loss: 2.1150
Epoch:   9 85.

Epoch:  11 53.3%     loss: 2.1007
Epoch:  11 54.1%     loss: 2.1146
Epoch:  11 54.9%     loss: 2.1224
Epoch:  11 55.7%     loss: 2.1053
Epoch:  11 56.5%     loss: 2.0976
Epoch:  11 57.3%     loss: 2.0897
Epoch:  11 58.1%     loss: 2.1050
Epoch:  11 58.9%     loss: 2.1182
Epoch:  11 59.7%     loss: 2.1103
Epoch:  11 60.5%     loss: 2.0973
Epoch:  11 61.3%     loss: 2.0808
Epoch:  11 62.1%     loss: 2.1032
Epoch:  11 62.9%     loss: 2.1083
Epoch:  11 63.7%     loss: 2.0962
Epoch:  11 64.5%     loss: 2.0935
Epoch:  11 65.3%     loss: 2.1054
Epoch:  11 66.1%     loss: 2.1016
Epoch:  11 66.9%     loss: 2.1108
Epoch:  11 67.7%     loss: 2.1017
Epoch:  11 68.5%     loss: 2.0937
Epoch:  11 69.3%     loss: 2.1172
Epoch:  11 70.1%     loss: 2.1082
Epoch:  11 70.9%     loss: 2.0914
Epoch:  11 71.7%     loss: 2.0996
Epoch:  11 72.4%     loss: 2.0933
Epoch:  11 73.2%     loss: 2.0864
Epoch:  11 74.0%     loss: 2.0916
Epoch:  11 74.8%     loss: 2.1019
Epoch:  11 75.6%     loss: 2.0903
Epoch:  11 76.

Epoch:  13 47.8%     loss: 2.0791
Epoch:  13 48.6%     loss: 2.0686
Epoch:  13 49.4%     loss: 2.0716
Epoch:  13 50.2%     loss: 2.0649
Epoch:  13 51.0%     loss: 2.0833
Epoch:  13 51.7%     loss: 2.0810
Epoch:  13 52.5%     loss: 2.0668
Epoch:  13 53.3%     loss: 2.0688
Epoch:  13 54.1%     loss: 2.0884
Epoch:  13 54.9%     loss: 2.0845
Epoch:  13 55.7%     loss: 2.0822
Epoch:  13 56.5%     loss: 2.0655
Epoch:  13 57.3%     loss: 2.0637
Epoch:  13 58.1%     loss: 2.0720
Epoch:  13 58.9%     loss: 2.0951
Epoch:  13 59.7%     loss: 2.0822
Epoch:  13 60.5%     loss: 2.0688
Epoch:  13 61.3%     loss: 2.0586
Epoch:  13 62.1%     loss: 2.0734
Epoch:  13 62.9%     loss: 2.0731
Epoch:  13 63.7%     loss: 2.0718
Epoch:  13 64.5%     loss: 2.0640
Epoch:  13 65.3%     loss: 2.0728
Epoch:  13 66.1%     loss: 2.0739
Epoch:  13 66.9%     loss: 2.0779
Epoch:  13 67.7%     loss: 2.0763
Epoch:  13 68.5%     loss: 2.0655
Epoch:  13 69.3%     loss: 2.0923
Epoch:  13 70.1%     loss: 2.0815
Epoch:  13 70.

Epoch:  15 39.0%     loss: 2.0346
Epoch:  15 39.8%     loss: 2.0305
Epoch:  15 40.6%     loss: 2.0418
Epoch:  15 41.4%     loss: 2.0542
Epoch:  15 42.2%     loss: 2.0438
Epoch:  15 43.0%     loss: 2.0592
Epoch:  15 43.8%     loss: 2.0595
Epoch:  15 44.6%     loss: 2.0361
Epoch:  15 45.4%     loss: 2.0394
Epoch:  15 46.2%     loss: 2.0421
Epoch:  15 47.0%     loss: 2.0393
Epoch:  15 47.8%     loss: 2.0539
Epoch:  15 48.6%     loss: 2.0457
Epoch:  15 49.4%     loss: 2.0479
Epoch:  15 50.2%     loss: 2.0458
Epoch:  15 51.0%     loss: 2.0569
Epoch:  15 51.7%     loss: 2.0524
Epoch:  15 52.5%     loss: 2.0478
Epoch:  15 53.3%     loss: 2.0442
Epoch:  15 54.1%     loss: 2.0654
Epoch:  15 54.9%     loss: 2.0631
Epoch:  15 55.7%     loss: 2.0523
Epoch:  15 56.5%     loss: 2.0399
Epoch:  15 57.3%     loss: 2.0327
Epoch:  15 58.1%     loss: 2.0497
Epoch:  15 58.9%     loss: 2.0700
Epoch:  15 59.7%     loss: 2.0565
Epoch:  15 60.5%     loss: 2.0417
Epoch:  15 61.3%     loss: 2.0460
Epoch:  15 62.

Epoch:  17 31.8%     loss: 2.0206
Epoch:  17 32.6%     loss: 2.0371
Epoch:  17 33.4%     loss: 2.0156
Epoch:  17 34.2%     loss: 2.0225
Epoch:  17 35.0%     loss: 2.0222
Epoch:  17 35.8%     loss: 2.0201
Epoch:  17 36.6%     loss: 2.0173
Epoch:  17 37.4%     loss: 2.0278
Epoch:  17 38.2%     loss: 2.0190
Epoch:  17 39.0%     loss: 2.0125
Epoch:  17 39.8%     loss: 2.0060
Epoch:  17 40.6%     loss: 2.0232
Epoch:  17 41.4%     loss: 2.0320
Epoch:  17 42.2%     loss: 2.0250
Epoch:  17 43.0%     loss: 2.0402
Epoch:  17 43.8%     loss: 2.0372
Epoch:  17 44.6%     loss: 2.0182
Epoch:  17 45.4%     loss: 2.0162
Epoch:  17 46.2%     loss: 2.0316
Epoch:  17 47.0%     loss: 2.0274
Epoch:  17 47.8%     loss: 2.0377
Epoch:  17 48.6%     loss: 2.0226
Epoch:  17 49.4%     loss: 2.0347
Epoch:  17 50.2%     loss: 2.0289
Epoch:  17 51.0%     loss: 2.0394
Epoch:  17 51.7%     loss: 2.0408
Epoch:  17 52.5%     loss: 2.0248
Epoch:  17 53.3%     loss: 2.0328
Epoch:  17 54.1%     loss: 2.0541
Epoch:  17 54.

# Make Predictions

In [14]:
def sample(inputs, hidden, topk=5):
    logits, hidden = model(inputs, hidden)
    
    last_output = logits[0, -1]              # keep last seq. output        shape [n_out]
    probs = F.softmax(last_output, dim=0)    # logits to probabilities      shape [n_out]
    
    probs, indices = probs.topk(topk)
    weights = probs / probs.sum()                             # normalize probs
    tok = np.random.choice(a=indices.detach().cpu().numpy(),  # no torch impl. yet :(
                           p=weights.detach().cpu().numpy())
    
    res = torch.tensor([[tok]], device=inputs.device)     # feed this to next sample() call
    return res, hidden

In [42]:
prime = 'KING:'
size = 5000

In [43]:
model.eval()

inputs = [c2i[c] for c in prime]                   # tokenize
inputs = torch.tensor(inputs).to(device)
inputs = inputs.reshape(1, -1)                     # shape [n_batch=1, n_seq]

result = []

with torch.no_grad():
    output, hidden = sample(inputs, None, topk=5)
    result.append(output.item())
    
    for i in range(size-1):
        output, hidden = sample(output, hidden, topk=5)
        result.append(output.item())

In [45]:
print(prime)
print(''.join([i2c[i] for i in result]))

KING:

Wourg my lation tens my honours on the streen.

CALSES:
Ale moran him man the the sorest to the sore.

DUCUS:
I wirl the to her honour hount of that shast me
If some a thall thou wardicitate the with
And some thou she trate a stave me the have the morad
And sengerise in a some, the worthy thas,
And as mestand the tall, this stromes hing
Worly the merest then have heal, and soure, the heir of home him,
What and this a head the talle mirth a thing the merter.
I'll why are see hite all thou
And wather then the, shall he see hens thou
shall serness a to shall thee as trith think of to stire, the
chordes the more saverers to has,
And the to this sones and as teer how,
But there alr ome and too to her and thy warled.

PORD:
In he the man mare and should thy sore.

DUKE ERTERCO:
I wall wites she than the my walles.

CASSILIUS:
That this wather hing of her shick of my
Thou this; where and a husces of thou she sissed this.

DECENAS:
To me, my thou shere havas me sille, we to me seed
Arar